In [ ]:
!pip install transformers torch

In [ ]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

In [ ]:
df = pd.read_csv("geo_facts.csv")
df.info()

In [ ]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [ ]:
tqdm.pandas()

In [ ]:
def get_relevance_score(location, sentence):
    candidate_labels = [f"related to {location}", f"not related to {location}"]
    result = classifier(sentence, candidate_labels)
    return result["scores"][result["labels"].index(f"related to {location}")]

In [ ]:
for i in range(1, 6):
    col = f"sentence {i}"
    df[f"relevance_score_{i}"] = df.progress_apply(lambda row: get_relevance_score(row["location"], row[col]), axis=1)

In [ ]:
df["location_relevance_score"] = df[[f"relevance_score_{i}" for i in range(1, 6)]].mean(axis=1)

In [ ]:
df.to_csv("geo_facts-zeroshot.csv", index=False)

In [ ]:
df[["location", "location_relevance_score"]].head()